# Step 0: Initialize project

Initialize `dataiku` client and project to the default one (i.e. the current one).

In [0]:
import dataiku

dss_client = dataiku.api_client()
project = dss_client.get_default_project()

Confirm project key:

In [0]:
project.project_key

# Step 1: Delete old plugin recipes

WARNING: Do not use this if you manually added or modified plugin recipes.

The block below deletes recipes whose types are the ones of the old `editable-via-webapp` plugin. This is ok when those recipes had been created automatically by the plugin's Visual Webapp: they will be automatically recreated when we update and restart the Webapp at Step 3, based on the new plugin.

In [0]:
for recipe in project.list_recipes(as_type="objects"):
    # print(recipe)
    recipe_type = recipe.get_settings().type
    if (
        recipe_type == "CustomCode_merge-edits"
        or recipe_type == "CustomCode_pivot-editlog"
    ):
        recipe.delete()

# Step 2: Rename datasets

Rename all datasets ending with `_editlog_pivoted` to end with `_edits`:

In [0]:
for d in project.list_datasets():
    dataset_name = d["name"]
    dataset = project.get_dataset(dataset_name)
    # if the name ends with '_editlog_pivoted', rename it to change the suffix to '_edits'
    if dataset_name.endswith("_editlog_pivoted"):
        new_name = dataset_name.replace("_editlog_pivoted", "_edits")
        # add to list of datasets to build
        dataset.rename(new_name)
        print(f"Renamed {dataset_name} to {new_name}")

# Step 3: Change Visual Webapps' type

- Stop the "data editing" Visual Webapps provided by the old `editable-via-webapp` plugin;
- Change their type to the one provided by the new `visual-edit` plugin;
- Restart them: this re-creates the recipes we deleted but with the types from the new plugin.

In [0]:
for webapp in project.list_webapps(as_type="objects"):
    webapp_settings = webapp.get_settings()
    if (
        webapp_settings.get_raw()["type"]
        == "webapp_editable-via-webapp_edit-dataset-records"
    ):
        webapp.stop_backend()
        webapp_settings.get_raw()["type"] = "webapp_visual-edit_visual-edit"
        webapp_settings.save()
        print(
            f"Changed type of {webapp_settings.get_raw()['name']} to 'webapp_visual-edit_visual-edit'"
        )
        webapp.start_or_restart_backend()